In [ ]:
import os
import shutil
import time

import astropy.units as u
import numpy as np
import pandas as pd
import astropy.constants as c

#Import the necessary path to draw the single pad transmon design. 
import sys
sys.path.append('Customized_Components')
from rounded_single_pad import Round_TransmonPocket_Single as transmon
import Transmon_property as trans_p
import Transmon_specifications as jj
from dolan_junction import DolanJunction as junction
import Default_params as dp
from pocket import TransmonPocket as pocket

from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal import MetalGUI, Dict
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

gui = MetalGUI(dp.design)
design = dp.design


In [ ]:
from scipy.optimize import brentq

In [46]:
test_path = r'C:\Users\slab\Desktop\Wendy-qiskit-code\data\educated_guess_test_test.csv'

In [43]:
default_options = dict(
        pos_x = '0mm', 
        pos_y = '0mm', 
        orientation = '0',
        frequency = 5,
        guess_path = test_path,
        coupling_path = '',
        coord = '(0,0)',
        qubit_layer = 5,
        junction_layer = 2, 
        ab_layer = 8,
        ab_square_layer = 9,
        ab_distance = '70um',
        rotation = 0
        )

In [41]:
guess_path = r'C:\Users\slab\Desktop\Wendy-qiskit-code\data\educated_guess_test2'
guess = pd.read_csv(guess_path, index_col=False).drop(columns=['Unnamed: 0'])

In [47]:
test = pd.read_csv(test_path).drop(columns=['Unnamed: 0'])

In [48]:
ljs = test['Ljs (nH)'].values

In [49]:
ljs

array([13, 13, 10, 10,  8,  8,  8,  6,  6,  6], dtype=int64)

In [52]:
design.delete_all_components()

In [53]:
gui.rebuild()

In [54]:
default_options['coord'] = '_0_0_'
default_options['frequency'] = 4.6
default_options['guess_path'] = test_path
# change_size(26, 0)

p = pocket(gui,design,sim = True, options = default_options)
gui.rebuild()

tried1


In [ ]:
p.resonators

In [ ]:
components = ['qubit_0_0_', 'TQ_0_0_', 'cpw__0_0_']

In [ ]:
from qiskit_metal.analyses.quantization import LOManalysis
from qiskit_metal.analyses.quantization import EPRanalysis

In [ ]:
c = LOManalysis(design, "q3d")
c.sim.setup.min_passes  = 10
c.sim.setup.max_passes = 20
c.sim.setup.freq_ghz = 5

c.sim.renderer.options['wb_threshold'] ='72um'
c.sim.renderer.options['x_buffer_width_mm'] = 0.5
c.sim.renderer.options['y_buffer_width_mm'] = 0.5

h = 0.4

c.sim.renderer.options['wb_height'] = h

In [ ]:
eig_all = EPRanalysis(design, 'hfss')

hfss = eig_all.sim.renderer
eig_all.sim.renderer.options['wb_threshold'] = '72um'

eig_all.sim.setup.max_passes = 30
eig_all.sim.setup.max_delta_f = 0.1
eig_all.sim.setup.n_modes = 2
eig_all.sim.renderer.options['x_buffer_width_mm'] = 0.5
eig_all.sim.renderer.options['y_buffer_width_mm'] = 0.5

In [ ]:
guess['Coupling_len']

In [ ]:
guess['Coupling_len(um)']

In [ ]:
guess

In [ ]:
def change_size(size, i, path  = test_path):
    dat = pd.read_csv(path, index_col=False).drop(columns=['Unnamed: 0'])
    sizes = list(dat['Coupling_len(um)'])
    sizes_str = list(dat['Coupling_len'])
    
    sizes[i] = size
    sizes_str[i] = str(size)+'um'
    
    dat.drop(columns = ['Coupling_len(um)','Coupling_len'],axis= 1, inplace = True)
    
    dat['Coupling_len(um)'] = sizes
    dat['Coupling_len'] = sizes_str
    
    dat.to_csv(path)

In [ ]:
p = pocket(gui,design,sim = True, options = default_options)
gui.rebuild()

In [ ]:
def find_freq(size, ii, path = test_path, p = p):
    change_size(size, ii, path = path)
    print(size)
    
    dat = {}
    target_freq = 4+0.2*ii
    dat['f_target'] = 4+0.2*ii
    default_options['frequency'] = target_freq
    dat['size'] = size
    dat['Lj'] = ljs[ii]
    
    file_path = r'C:\Users\slab\Desktop\Wendy-qiskit-code\data\optimize_cp_len{}.csv'.format(target_freq)
    try:
        datas = pd.read_csv(file_path, index_col=False).drop(columns=['Unnamed: 0'])
    except:
        datas = pd.DataFrame()
    
    
    
    eig_all.sim.renderer.options['Lj'] = str(ljs[ii])+'nH'
    eig_all.sim.renderer.options['Cj'] = str(jj.find_junction_capacitance((ljs[ii]*u.nH)))

    eig_all.sim.setup.vars = {}
    eig_all.sim.setup.vars['Lj'] = str(ljs[ii])+'nH'
    eig_all.sim.setup.vars['Cj'] = str(jj.find_junction_capacitance((ljs[ii]*u.nH)))

    p = pocket(gui,design,sim = True, options = default_options)
    gui.rebuild()
    
    eig_all.sim.run(name="single_q", components=components)
    
    
    convergence = pd.read_csv('hfss_eig_f_convergence.csv')
    conv = convergence.dropna()

    ind = list(conv.keys())[1:]
    for i in range(2):
        freq = conv[ind[(i)]].values[-1]
        dat['Freq'+str(i+1)+'(GHz)'] = freq
    
    
    try:
        del eig_all.setup.junctions['jj']
    except:
        print('saddd')
    eig_all.setup.junctions.jj1 = Dict(rect='JJ_rect_Lj_qubit_0_0__rect_jj', line='JJ_Lj_qubit_0_0__rect_jj_',
                    Lj_variable='Lj', Cj_variable='Cj')
    eig_all.run_epr()
    
    #Save the EPR data
    chi = eig_all.sim.renderer.epr_quantum_analysis.get_chis().to_numpy()
    freq_EPR= eig_all.sim.renderer.epr_quantum_analysis.get_frequencies().to_numpy()[:,0]
    
    # renderer_hfss.clean_active_design()
    for i in range(2):
        freq = freq_EPR[i]
        dat['Freq_EPR'+str(i+1)+'(MHz)'] = freq
        for j in range(i+1):
            dat['Chi'+str(i+1)+'_'+str(j+1)+'(MHz)'] = chi[i][j] 

    
    # c.sim.run(name = 'a',components=components)
    # cg = -c.sim.capacitance_matrix['a_connector_pad_qubit_0_0_']['pad_top_qubit_0_0_']
    # dat['cap_t_g'] = cg
    # f,_ = trans_p.transmon_freq(cg*u.fF, ljs[i]*u.nH)
    
    # dat['Predict_freq'] = f
    
    
    data = pd.DataFrame(dat, index = [0])
    
    datas = pd.concat([datas,data], ignore_index = True)
    datas.to_csv(file_path)
    
    return chi[0][1]-1

In [ ]:
test

optimization start

In [56]:
for i in [5]:
    
    smin, smax= (20,100)
    
    def f(size):
        return find_freq(size, i)
    
    sfinal = brentq(f, smin, smax, xtol = 1e-2, rtol = 1e-2)
    
    
    change_size(sfinal,i, path = guess_path)
    
    
    

20.0
tried1


INFO 11:49PM [connect_design]: 	Opened active design
	Design:    single_q_hfss [Solution type: Eigenmode]
INFO 11:51PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 11:51PM [analyze]: Analyzing setup Setup
12:03AM 23s INFO [get_f_convergence]: Saved convergences to c:\Users\slab\Desktop\Wendy-qiskit-code\hfss_eig_f_convergence.csv


saddd
Design "single_q_hfss" info:
	# eigenmodes    2
	# variations    1
Design "single_q_hfss" info:
	# eigenmodes    2
	# variations    1

        energy_elec_all       = 6.85684180844162e-25
        energy_elec_substrate = 6.21761425756444e-25
        EPR of substrate = 90.7%

        energy_mag    = 6.85295352919937e-25
        energy_mag % of energy_elec_all  = 99.9%
        

Variation 0  [1/1]

  Mode 0 at 5.18 GHz   [1/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               99.2%  1.888e-25 1.561e-27

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 98.33%
	jj1              1.29265  (+)        0.0218945
		(U_tot_cap-U_tot_ind)/mean=-12.01%
Calculating Qdielectric_main for mode 0 (0/1)


  sol = sol.append(self.get_Qdielectric(



p_dielectric_main_0 = 0.9176919895464821

  Mode 1 at 7.52 GHz   [2/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
                0.1%  3.428e-25 3.426e-25

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_1j   sign s_1j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 96.55%
	jj1             0.00111332  (+)        3.97742e-05
		(U_tot_cap-U_tot_ind)/mean=-0.03%
Calculating Qdielectric_main for mode 1 (1/1)


...

Lj,8
0,4983.71
1,7462.97


#### Kerr Non-linear coefficient table (MHz)

###### Numerical diagonalization

0         1
Lj                    
8  0  171.18  9.98e-01
   1    1.00  1.59e-03

12:54AM 21s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  CRITICAL: QEventDispatcherWin32::wakeUp: Failed to post a message (Not enough quota is available to process this command.)

12:54AM 21s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  CRITICAL: QEventDispatcherWin32::wakeUp: Failed to post a message (Not enough quota is available to process this command.)



end

Sort data

In [ ]:
data = pd.read_csv(guess_path)

In [ ]:
data

In [ ]:
cols = data.columns.values
f = []
for i in cols:
    if 'Unnamed' not in i:
        f.append(i)

In [ ]:
d = data[f]

In [ ]:
d.to_csv(r'data\educated_guess_test2')

sim start

In [ ]:
datas = pd.DataFrame()

for ii,freq in enumerate(np.arange(4,6,0.2)):
    dat = {}
    dat['f_target'] = freq
    default_options['frequency'] = freq
    
    q1 = p.qubit
    
    eig_all.sim.renderer.options['Lj'] = ljs[ii]
    eig_all.sim.renderer.options['Cj'] = q1.options['hfss_capacitance']

    eig_all.sim.setup.vars = {}
    eig_all.sim.setup.vars['Lj'] = q1.options['hfss_inductance']
    eig_all.sim.setup.vars['Cj'] = q1.options['hfss_capacitance']

    p = pocket(gui,design,sim = True, options = default_options)
    gui.rebuild()
    
    eig_all.sim.run(name="single_q", components=components)
    
    
    convergence = pd.read_csv('hfss_eig_f_convergence.csv')
    conv = convergence.dropna()

    ind = list(conv.keys())[1:]
    for i in range(1):
        freq = conv[ind[(i)]].values[-1]
        dat['Freq'+str(i+1)+'(GHz)'] = freq
    
    
    try:
        del eig_all.setup.junctions['jj']
    except:
        print('saddd')
    eig_all.setup.junctions.jj1 = Dict(rect='JJ_rect_Lj_qubit_0_0__rect_jj', line='JJ_Lj_qubit_0_0__rect_jj_',
                    Lj_variable='Lj', Cj_variable='Cj')
    eig_all.run_epr()
    
    #Save the EPR data
    chi = eig_all.sim.renderer.epr_quantum_analysis.get_chis().to_numpy()
    freq_EPR= eig_all.sim.renderer.epr_quantum_analysis.get_frequencies().to_numpy()[:,0]
    
    # renderer_hfss.clean_active_design()
    for i in range(1):
        freq = freq_EPR[i]
        dat['Freq_EPR'+str(i+1)+'(MHz)'] = freq
        for j in range(i+1):
            dat['Chi'+str(i+1)+'_'+str(j+1)+'(MHz)'] = chi[i][j] 

    
    c.sim.run(name = 'a',components=components)
    cg = -c.sim.capacitance_matrix['a_connector_pad_qubit_0_0_']['pad_top_qubit_0_0_']
    dat['cap_t_g'] = cg
    f,_ = trans_p.transmon_freq(cg*u.fF, ljs[ii]*u.nH)
    
    dat['Predict_freq'] = f
    
    
    data = pd.DataFrame(dat, index = [0])
    
    datas = pd.concat([datas,data], ignore_index = True)
    datas.to_csv(r'data\epr_freq_cap_freq.csv')

In [ ]:
i